## Upotrebom procesa koji simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke

Svaki proces je simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada implementirati poseban servis (dodatni proces) kojem sve ćelije javljaju novo stanje prilikom promene (pri čemu poruke sadrže koordinate ćelije, broj iteracije i novo stanje). Servis treba da rekonstruiše i sačuva (ili vrati u glavni program) niz matrica stanja.

In [1]:
# Imports: #
import random, multiprocessing, time, queue
import numpy as np

In [2]:
# Global variables: 

# Safe to change values: 
n_epoch = 50                        # Number of epochs ( iterations ) Conway's game of life have.
n = 30                              # Number of rows and columns ( matrix type: NxN ).

# Not safe to change values: 
n_neighbours = 8                    # Number of cell neighbours ( TOP_LEFT, TOP, TOP_RIGHT, LEFT, RIGHT, BOT_LEFT, BOT, BOT_RIGHT ). 

# Global structs: 
cells = None
steps = None                        # List of matrixes trough epoches - list[i] is the appereance of the values of the every cell state in the i-th epoch.
epoch_condition = multiprocessing.Condition()
cells_left = multiprocessing.Value( 'i', n ** 2, lock = True )

queues = list()
write_mutex = multiprocessing.Lock()
queue_mutex = multiprocessing.Lock()

queue_matrix = [ [ multiprocessing.Queue() for i in range( n ) ] for j in range( n ) ]
cell_info_queue = multiprocessing.Queue()

In [3]:
# SavingProcess 

class SavingProcess( multiprocessing.Process ):

    def __init__( self, initial_state, cell_info_queue ):
        
        super().__init__()
       
        self.initial_state = initial_state
        self.cell_info_queue = cell_info_queue
        self.step_queue = multiprocessing.Queue()
        

    def reconstruct_steps( self ):
        
        ret_steps = list()

        for _ in range( n_epoch + 1 ):
            ret_steps.append( self.step_queue.get() )  
                    
        return ret_steps

    def run( self ):

        saved_steps = list()
        saved_steps.append( self.initial_state )

        for _ in range( n_epoch ):
            for _ in range( n**2 ):

                ( x, y, epoch, state ) = cell_info_queue.get()
                epoch += 1 
                
                try:
                    saved_steps[ epoch ][x][y] = state 
                except IndexError:
                    self.step_queue.put( saved_steps[ epoch - 1 ] )
                    saved_steps.append( np.zeros( shape = ( n, n ), dtype = int ) )
                    saved_steps[ epoch ][x][y] = state
       
        self.step_queue.put( saved_steps.pop() )

In [4]:
# Cell 

class Cell( multiprocessing.Process ):
    
    def __init__( self, x, y, queue_matrix, cell_info_queue, queues ):
        
        super().__init__()

        self.state = random.randint( 0, 1 )             # Generating random state for first epoch. 
        self.epoch = 0                                  # Current epoch ( iteration ). 

        self.x = x                                      # The x coordinate of cell in the matrix. 
        self.y = y                                      # The y coordinate of cell in the matrix. 
        
        self.queue_matrix = queue_matrix
        self.cell_info_queue = cell_info_queue
        self.queues = queues


    def __str__( self ):
           return f'[ { self.x } ][ { self.y } ] - { self.state }'


    def notify_neighbours_queues( self ):
        global queue_matrix
        
        x = self.x
        y = self.y  
        state = self.state

        queue_matrix[ ( x - 1 ) % n ][ ( y - 1 ) % n ].put( state )             # TOP LEFT
        queue_matrix[ ( x - 1 ) % n ][ y ].put( state )                         # TOP
        queue_matrix[ ( x - 1 ) % n ][ ( y + 1 ) % n ].put( state )             # TOP RIGHT
        queue_matrix[ x ][ ( y - 1 ) % n ].put( state )                         # LEFT      
        queue_matrix[ x ][ ( y + 1 ) % n ].put( state )                         # RIGHT
        queue_matrix[ ( x + 1 ) % n ][ ( y - 1 ) % n ].put( state )             # BOT LEFT
        queue_matrix[ ( x + 1 ) % n ][ y ].put( state )                         # BOT
        queue_matrix[ ( x + 1 ) % n ][ ( y + 1 ) % n ].put( state )             # BOT RIGHT


    def run( self ):
        
        global cells, cells_left, is_saved, cell_info_queue, queue_matrix

        # Every thread has to loop n_epoch times.
        for epoch in range( 0, n_epoch ):          
            
            self.notify_neighbours_queues()

            alive = 0

            for _ in range( 0, n_neighbours ):
                alive += queue_matrix[ self.x ][ self.y ].get()        
            
            # Cell is alive.
            if self.state == 1:
                if alive < 2 or alive > 3:
                    self.state = 0
            
            # Cell is dead.
            else:
                if alive == 3:
                    self.state = 1

            # Sending state to SavingProcess.
            cell_info = ( self.x, self.y, epoch, self.state )
            self.cell_info_queue.put( cell_info )

            epoch_condition.acquire()     
              
            # Operations like -= which involve a read and write are not atomic.
            with cells_left.get_lock():
                cells_left.value -= 1      
    
            if cells_left.value == 0:
                cells_left.value = n ** 2
                epoch_condition.notify_all()    # notify_all() Does NOT release Condition Lock.
            else:
                epoch_condition.wait()          # wait() releases the Condition Lock, and re-aquires it when woken.
            
            epoch_condition.release()

            self.epoch = epoch

In [5]:
# Main

def main():

    global cells, steps, cells_left, steps

    # Generating cells with random state.
    cells = [ [ Cell( i, j, queue_matrix, cell_info_queue, queues ) for j in range( n ) ] for i in range( n ) ]
    
    # Generating initial step matrix with cell states.
    initial_state = np.array( [ [ cells[i][j].state for j in range( n ) ] for i in range( n ) ] )
    
    saving_process = SavingProcess( initial_state, cell_info_queue )
    saving_process.start()
    
    for i in range( n ):
        for j in range( n ):
            cells[i][j].start()

    for i in range( n ):
        for j in range( n ):
            cells[i][j].join()
        
    steps = saving_process.reconstruct_steps()
    # saving_process.join()
    
    
if __name__ == "__main__":
    main()

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

def animate( steps ):
  '''
  Animates the array of matrices - each matrix is a single state in simulation.
  '''
  
  def init():
    im.set_data( steps[0] )
    return [ im ]
  

  def animate( i ):
    im.set_data( steps[i] )
    return [ im ]


  im = plt.matshow( steps[0], interpolation = 'None', animated = True );
  
  anim = FuncAnimation( im.get_figure(), animate, init_func = init,
                  frames = len( steps ), interval = 1000, blit = True, repeat = False );
                  
  return anim

anim = animate( steps );
HTML( anim.to_html5_video() )